In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacker_cluster').getOrCreate()

In [2]:
data = spark.read.csv('/FileStore/tables/hack_data.csv', inferSchema=True, header=True)

In [3]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [4]:
data = data.drop('location')

In [5]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| 72.37|
 20.0| 720.99| 0| 3.04| 9.0| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| 72.03|
 15.0| 275.53| 1| 3.53| 8.0| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| 69.99|
 15.0| 249.09| 1| 3.39| 9.0| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| 68.56|
 9.0| 284.77| 0| 3.12| 9.0| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
only showing top 20 rows

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')
final_data = assembler.transform(data)

In [8]:
from pyspark.ml.feature import StandardScaler

In [9]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [10]:
scaler_model = scaler.fit(final_data)

In [11]:
final_data = scaler_model.transform(final_data)

In [12]:
from pyspark.ml.clustering import KMeans

In [13]:
# Since we are not sure where there were 2 or 3 hackers, we'll make 2 models
kmeans_2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans_3 = KMeans(featuresCol='scaledFeatures', k=3)

In [14]:
model_2 = kmeans_2.fit(final_data)
model_3 = kmeans_3.fit(final_data)

In [15]:
print('WSSSE')
print(model_2.computeCost(final_data))
print(model_3.computeCost(final_data))

WSSSE
601.7707512676716
434.1492898715845

In [16]:
centers_2 = model_2.clusterCenters()
centers_2

# 2 distinct centroids so seems like 2 hackers

Out[26]: [array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612])]

In [17]:
centers_3 = model_3.clusterCenters()
centers_3

# Here the 2nd and 3rd centroids are very close to one another. This confirms that were 2 hackers

Out[27]: [array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ]),
 array([1.21780112, 1.37901802, 1.99757683, 1.37198977, 2.55237797,
 5.29152222]),
 array([1.30217042, 1.25830099, 0. , 1.35793211, 2.57251009,
 5.24230473])]

In [18]:
# The class split is even which means 2 hackers
model_2.transform(final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [19]:
# The class splits are not even which means there weren't 3 hackers
model_3.transform(final_data).select('prediction').groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+